Obtain Johns Hopkins University CSSE COVID data from BigQuery and transform and enhance it for plotting.  Write the result back to BigQuery as a table.

Based on the excellent tutorial here: https://towardsdatascience.com/covid-19-data-processing-58aaa3663f6

In [1]:
import pandas as pd
import numpy as np

import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage
import google.cloud.bigquery.magics

# use BigQuery Storage API by default in BigQuery queries
google.cloud.bigquery.magics.context.use_bqstorage_api = True

In [2]:
# Explicitly create a credentials object. This allows you to use the same
# credentials for both the BigQuery and BigQuery Storage clients, avoiding
# unnecessary API calls to fetch duplicate authentication tokens.
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Make clients.
bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [3]:
query_string = """
    SELECT *
    FROM `bigquery-public-data.covid19_jhu_csse.confirmed_cases`
    ORDER BY country_region
"""

cases_df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
#print(cases_df.head())
cases_df = cases_df.drop(columns=['latitude', 'longitude', 'location_geom'])
#print(cases_df.columns)

query_string = """
    SELECT *
    FROM `bigquery-public-data.covid19_jhu_csse.deaths`
    ORDER BY country_region
"""

deaths_df = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
#print(deaths_df.head())
deaths_df = deaths_df.drop(columns=['latitude', 'longitude', 'location_geom'])
#print(deaths_df.columns)

# replace both dataframe's columns with reformatted dates

# Fix the date formatting in the column headers
cols = cases_df.columns
new_cols = ['Province/State', 'Country']
for col in cols:
    if col.startswith('_'):
        # original format is _MM_DD_YY, desired format  is MM/DD/YY
        new_col = col.replace('_', '/')[1:]
        new_cols.append(new_col)  

cases_df.columns = new_cols
deaths_df.columns = new_cols

#print(cases_df.head())
#print(deaths_df.head())

In [4]:
#Transpose
dates = cases_df.columns[3:]

cases_df_long = cases_df.melt(
    id_vars=['Province/State', 'Country'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Cases'
)

deaths_df_long = deaths_df.melt(
    id_vars=['Province/State', 'Country'], 
    value_vars=dates, 
    var_name='Date', 
    value_name='Deaths'
)

# Merging cases_df_long and deaths_df_long
full_table = cases_df_long.merge(
  right=deaths_df_long, 
  how='left',
  on=['Province/State', 'Country', 'Date']
)

# convert Date column from string to Date format
full_table['Date'] = pd.to_datetime(full_table['Date'])


# get rid of the rows pertaining to cruise ships
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | \
    full_table['Province/State'].str.contains('Diamond Princess') | \
    full_table['Country'].str.contains('Diamond Princess') | \
    full_table['Country'].str.contains('MS Zaandam')
    
full_table = full_table[~(ship_rows)]

#print(full_table.head())

  Province/State      Country     Date  Cases
0           None  Afghanistan  1/23/20      0
1           None      Albania  1/23/20      0
2           None      Algeria  1/23/20      0
3           None      Andorra  1/23/20      0
4           None       Angola  1/23/20      0
  Province/State      Country     Date  Deaths
0           None  Afghanistan  1/23/20       0
1           None      Albania  1/23/20       0
2           None      Algeria  1/23/20       0
3           None      Andorra  1/23/20       0
4           None       Angola  1/23/20       0


Next, let’s aggregate data into Country/Region wise and group them by Date and Country/Region.

In [5]:
full_grouped = full_table.groupby(['Date', 'Country'])[['Cases', 'Deaths']].sum().reset_index()

Now add day-wise Daily Cases and Daily Deaths by deducting the corresponding accumulative data on the previous day.

In [6]:
# new cases 
temp = full_grouped.groupby(['Country', 'Date', ])[['Cases', 'Deaths']]

temp = temp.sum().diff().reset_index()

mask = temp['Country'] != temp['Country'].shift(1)

temp.loc[mask, 'Cases'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan

# renaming columns
temp.columns = ['Country', 'Date', 'Daily Cases', 'Daily Deaths']

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country', 'Date'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['Daily Cases', 'Daily Deaths']
full_grouped[cols] = full_grouped[cols].astype('int')

# 
full_grouped['Daily Cases'] = full_grouped['Daily Cases'].apply(lambda x: 0 if x<0 else x)
full_grouped['Daily Deaths'] = full_grouped['Daily Deaths'].apply(lambda x: 0 if x<0 else x)

#print(full_grouped.tail())

Now compute the 14-day moving average for each jurisdiction

In [7]:
new_column = full_grouped.groupby('Country').rolling(14)['Daily Cases'].mean().round(0)
full_grouped['Cases 14d avg'] = new_column.reset_index(level=0, drop=True)

new_column = full_grouped.groupby('Country').rolling(14)['Daily Deaths'].mean().round(0)
full_grouped['Deaths 14d avg'] = new_column.reset_index(level=0, drop=True)

#full_grouped.tail()

In [8]:
full_grouped[full_grouped.Country=='Canada'].tail(n=40)

,Date,Country,Cases,Deaths,Daily Cases,Daily Deaths,Cases 14d avg,Deaths 14d avg
54464,2020-11-06,Canada,258862,10492,5393,61,3168.0,40.0
54653,2020-11-07,Canada,263208,10546,4346,54,3369.0,41.0
54842,2020-11-08,Canada,267152,10574,3944,28,3449.0,41.0
55031,2020-11-09,Canada,272023,10619,4871,45,3504.0,42.0
55220,2020-11-10,Canada,276468,10692,4445,73,3627.0,46.0
55409,2020-11-11,Canada,280452,10747,3984,55,3721.0,47.0
55598,2020-11-12,Canada,285926,10827,5474,80,3897.0,50.0
55787,2020-11-13,Canada,290696,10884,4770,57,3984.0,52.0
55976,2020-11-14,Canada,295126,10946,4430,62,4130.0,54.0
56165,2020-11-15,Canada,299427,11000,4301,54,4271.0,55.0
